Populate config for all the known networks

In [1]:
CHECKER_ADDR = '0xd59431D244F1B67925fB1Edd1EBa3d44348956b7'
CHECKER_ABI = '[{"inputs":[],"name":"check","outputs":[{"internalType":"bool","name":"result","type":"bool"}],"stateMutability":"view","type":"function"}]'

NETWORKS = {
    'Ethereum': 'https://ethereum.publicnode.com',
    'Binance Smart Chain': 'https://bsc.publicnode.com',
    'Optimism': 'https://optimism.publicnode.com',
    'Arbitrum One': 'https://arbitrum-one.publicnode.com',
    'Polygon': 'https://polygon-bor.publicnode.com',
    'Fantom': 'https://fantom.publicnode.com',
    'Avalanche': 'https://avalanche-c-chain.publicnode.com',
    'Gnosis Chain': 'https://gnosis.publicnode.com',
    'Aurora': 'https://mainnet.aurora.dev',
    'Base': 'https://base.publicnode.com',
    'Klaytn': 'https://1rpc.io/klay',
    'zkSync Era': 'https://mainnet.era.zksync.io',
}

CUSTOM_ADDRESSES = {
    'Klaytn': '0x7aB5194e730C4eC4a9d1d7f1DCAD9c173075888A',
    'zkSync Era': '0x3a23984F0E0dA1e8A5c4190D35d7be3e80409691',
}

Check all the known networks and populate `results` dictionary with data like this `{ 'Ethereum': True }`

In [2]:
from multiprocessing.pool import ThreadPool as Pool
from web3 import Web3

def is_push0_supported(network, url):
    w3 = Web3(Web3.HTTPProvider(url))
    checker = w3.eth.contract(address=CUSTOM_ADDRESSES[network] if network in CUSTOM_ADDRESSES else CHECKER_ADDR, abi=CHECKER_ABI)
    return network, checker.functions.check().call()

with Pool(10) as p:
    results = { result[0]: result[1] for result in p.starmap(is_push0_supported, NETWORKS.items()) }

Create html table using gathered data

In [3]:
prefix = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Is PUSH0 already supported?</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #ECF0F9; /* Light pastel blue background */
            color: #2C3E50; /* Darker text for better readability */
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }
        .container {
            max-width: 600px; /* Ensures table does not exceed the container's width */
            text-align: center;
        }
        table {
            border-collapse: collapse;
            width: 80%;
            margin: 20px auto; /* Center table */
        }
        td {
            text-align: left;
            padding: 8px;
            border: 1px solid #BDC3C7; /* Adjusted border color for visibility */
            color: #34495E; /* Even darker text for rows for contrast */
            font-size: 20px; 
        }
        .yes {
            background-color: #C2F0C2; /* Soft green, consider making it slightly lighter if needed */
            color: #27AE60; /* Dark green text for readability */
        }
        .no {
            background-color: #F0C2C2; /* Soft red, consider making it slightly lighter if needed */
            color: #C0392B; /* Dark red text for readability */
        }
        .bold {
            font-weight: bold; /* CSS for making text bold */
        }
        img.logo {
            width: 200px; /* Adjust size as needed */
            margin-bottom: -10px;
        }
    </style>
</head>
<body>

<div class="container">
    <img src="ethereum-shanghai-logo.webp" alt="Ethereum-Shanghai Logo" class="logo">
    <h1>Is PUSH0 already supported?</h1>
    <table>
"""

table = ''

for k, v in results.items():
    class_value = 'yes' if v else 'no'
    table_text = 'Yes' if v else 'No'
    table += f"""        <tr class="{class_value}">
            <td class="bold">{k}</td>
            <td>{table_text}</td>
        </tr>
"""

suffix = """
    </table>
</div>

</body>
</html>
"""

with open('docs/index.html', 'w') as out_f:
    print(prefix + table[:-1] + suffix, file=out_f)